In [1]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
if RunningInCOLAB:
    !git clone https://github.com/MJC598/Neuron_Burst_Analysis.git

In [2]:
%matplotlib notebook
import torch.nn as nn
import numpy as np
import torch
torch.manual_seed(0)

from torch.utils.data import TensorDataset, DataLoader
import scipy.io
import random
import pandas as pds
import time

from scipy import stats
from sklearn.metrics import r2_score
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### Class Explanations

These are 3 regression RNN-based models. In order to change it to a classifier the 
nn.Linear layers must have their second parameter changed to match the number of 
expected outputs.

* Expected Input Shape: (batch_size, time_sequence, features)

* Input_Size - number of features
* Hidden_Size - number of connections between the hidden layers
* Batch_Size - How many samples you want to push through the network before executing backprop
    (this is a hyperparameter that can change how fast or slow a model converges)
* Batch_First - Should always be set to True to keep input shape the same
* Dropout - Only really does anything with more than 1 layer on the LSTM, RNN, GRU. Useful to help generalize training

In [3]:
class baselineRNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(baselineRNN, self).__init__()
        self.rnn1 = nn.RNN(input_size=input_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)

    def forward(self, x):
        x, h_n  = self.rnn1(x,self.h0)

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out

class baselineLSTM(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(baselineLSTM, self).__init__()
        self.rnn = nn.LSTM(input_size=input_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)
        self.c0 = torch.randn(num_layers, batch_size, hidden_size)

    def forward(self, x):
        x, (h_n, c_n)  = self.rnn(x,(self.h0,self.c0))

        # take all outputs
        out = self.lin(x[:, -1, :])

        return out

class baselineGRU(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(baselineGRU, self).__init__()
        self.rnn = nn.GRU(input_size=input_size,hidden_size=hidden_size,
                          num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)

    def forward(self, x):
        # print(self.h0.shape)
        x, h_n  = self.rnn(x,self.h0)

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out
    
class conv1DLSTM(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(conv1DLSTM, self).__init__()
        self.c1 = nn.Conv1d(input_size, hidden_size, 3)
        self.p1 = nn.AvgPool1d(3)
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 2)
        self.p2 = nn.AvgPool1d(2)
        self.c3 = nn.Conv1d(hidden_size, hidden_size, 1)
#         self.p1 = nn.AvgPool1d(2)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)
        self.rnn = nn.LSTM(input_size=hidden_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)
        self.c0 = torch.randn(num_layers, batch_size, hidden_size)
        
    def forward(self, x):
        
        #switch (batch, sequence, feature) to (batch, feature, sequence)
        x = x.transpose(1,2)
        x = self.dropout(x)
        x = self.c1(x)
        x = self.p1(x)
#         x = self.tanh(x)
#         x = self.c2(x)
#         x = self.p2(x)
#         x = self.dropout(x)
        x = self.c3(x)
        x = self.sigmoid(x)
        
        #switch backwards
        x = x.transpose(1,2)
        x, (h_n, c_n)  = self.rnn(x,(self.h0,self.c0))

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out
    
class conv1DGRU(nn.Module):
    def __init__(self,input_size,hidden_size,output_size=1,
                 batch_size=1,num_layers=1,batch_first=True,dropout=0.0):
        super(conv1DGRU, self).__init__()
        self.c1 = nn.Conv1d(input_size, hidden_size, 5)
        self.p1 = nn.AvgPool1d(5)
        self.c2 = nn.Conv1d(hidden_size, hidden_size, 3)
        self.p2 = nn.AvgPool1d(3)
        self.c3 = nn.Conv1d(hidden_size, hidden_size, 1)
#         self.p1 = nn.AvgPool1d(2)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(p=0.2)
        self.rnn = nn.GRU(input_size=hidden_size,hidden_size=hidden_size,
                           num_layers=num_layers,batch_first=batch_first,dropout=dropout)
        self.lin = nn.Linear(hidden_size,output_size)
        self.h0 = torch.randn(num_layers, batch_size, hidden_size)
        
    def forward(self, x):
        
        #switch (batch, sequence, feature) to (batch, feature, sequence)
        x = x.transpose(1,2)
        x = self.dropout(x)
        x = self.c1(x)
        x = self.p1(x)
        x = self.c2(x)
#         x = self.p2(x)
#         x = self.c3(x)
        x = self.sigmoid(x)
        
        #switch backwards
        x = x.transpose(1,2)
        x, h_n  = self.rnn(x,self.h0)

        # take last cell output
        out = self.lin(x[:, -1, :])

        return out

In [4]:
TIMESTEPS = 49
FRONT_TIME = 0
BACK_TIME = -1
T_START = 50+FRONT_TIME
T_END = 50+BACK_TIME
MODEL = baselineLSTM
OUTPUT = '1ms_lag_FR'
LOSS_FILE = ('losses/bursts/losses_' + str(MODEL) + 
             '_' + OUTPUT + '_' + str(FRONT_TIME) + 
             '_' + str(T_END) + '_fullin.csv')
PATH = ('models/bursts/' + str(MODEL) + '_' + OUTPUT + 
        '_' + str(FRONT_TIME) + '_' + str(T_END) + 
        '_fullin.pth')
DATA_PATH = 'data/bursts/burst_separatePNITNv2.mat'
COLAB_PRE = 'Neuron_Burst_Analysis/'
if RunningInCOLAB:
    LOSS_FILE = COLAB_PRE + LOSS_FILE
    PATH = COLAB_PRE + PATH
    DATA_PATH = COLAB_PRE + DATA_PATH

# Specific Model Parameters
input_size = 9
hidden_size = 32
output_size = TIMESTEPS
batch_size = 32
num_layers = 1
batch_first = True
dropout = 0.0
epochs = 75

In [5]:
def get_data_from_mat(file_path, output_index=None, type='pre_pn'):
    data = scipy.io.loadmat(file_path)
    duration = []
    amp = []
    pb_fr_pn = []
    pb_fr_itn = []
    pb_aff_pn = []
    pb_aff_itn = []
    pb_exc_pn = []
    pb_inh_pn = []
    pb_exc_itn = []
    pb_inh_itn = []
    pb_lfp = []
    wb_fr_pn = []
    wb_fr_itn = []
    wb_aff_pn = []
    wb_aff_itn = []
    wb_exc_pn = []
    wb_inh_pn = []
    wb_exc_itn = []
    wb_inh_itn = []
    wb_lfp = []

#     print(data['info_collect'][0])
#     print(data['info_collect'].shape)
    for i in range(1, data['info_collect'].shape[0]):
        arr = data['info_collect'][i]
        
#         print(arr.shape)
        
        duration.append(arr[0])
        amp.append(arr[1])
        
        pb_fr_pn.append(arr[2])
        pb_fr_itn.append(arr[3])
        pb_aff_pn.append(arr[4])
        pb_aff_itn.append(arr[5])
        pb_exc_pn.append(arr[6])
        pb_inh_pn.append(arr[7])
        pb_exc_itn.append(arr[8])
        pb_inh_itn.append(arr[9])
        pb_lfp.append(arr[10])
        
#         print(arr[10].shape)
        wb_fr_pn.append(arr[11])
        wb_fr_itn.append(arr[12])
        wb_aff_pn.append(arr[13])
        wb_aff_itn.append(arr[14])
        wb_exc_pn.append(arr[15])
        wb_inh_pn.append(arr[16])
        wb_exc_itn.append(arr[17])
        wb_inh_itn.append(arr[18])
        wb_lfp.append(arr[19])
    
#     print(np.array(pb_fr_pn).shape)
#     print(np.array(pb_fr_itn).shape)
#     print(np.array(pb_aff_pn).shape)
#     print(np.array(pb_aff_itn).shape)
#     print(np.array(pb_exc_pn).shape)
#     print(np.array(pb_inh_pn).shape)
#     print(np.array(pb_exc_itn).shape)
#     print(np.array(pb_inh_itn).shape)
    
#     print(np.array(wb_fr_pn).shape)
#     print(np.array(wb_fr_itn).shape)
#     print(np.array(wb_aff_pn).shape)
#     print(np.array(wb_aff_itn).shape)
#     print(np.array(wb_exc_pn).shape)
#     print(np.array(wb_inh_pn).shape)
#     print(np.array(wb_exc_itn).shape)
#     print(np.array(wb_inh_itn).shape)
    
    full_labels = np.concatenate((pb_fr_pn, pb_fr_itn), axis=2)
    
    front_data = np.concatenate((pb_fr_pn, pb_fr_itn, pb_aff_pn, pb_aff_itn,pb_exc_pn, 
                                 pb_inh_pn, pb_exc_itn, pb_inh_itn, pb_lfp), axis=2)
    
#     rear_data = np.concatenate((wb_fr_pn, wb_fr_itn, wb_aff_pn, wb_aff_itn,
#                                 wb_exc_pn, wb_inh_pn, wb_exc_itn, wb_inh_itn), axis=2)
    
    full_data = front_data
    
    for j in range(2):
        x = full_labels[:,:,j]
        full_labels[:,:,j] = (x - np.min(x))/(np.max(x)-np.min(x))
    
    for i in range(full_data.shape[0]):
        for j in range(input_size):
            x = full_data[i,:,j]
            full_data[i,:,j] = (x - np.min(x))/(np.max(x)-np.min(x))
    
#     print(full_data.shape)
    
    random.seed(10)
    data_samples = 5472 #5498 
    k = 4352
    full = np.arange(data_samples)
    training_indices = np.random.choice(full, size=k, replace=False)
    validation_indices = np.delete(full,training_indices)
    
    training_data = full_data[:k,0:49,:] 
    validation_data = full_data[k:data_samples,0:49,:]
    
    if output_index is None:
        training_labels = full_labels[:k,1:50,:] 
    else:
        training_labels = full_labels[:k,1:50,output_index]
    
    if output_index is None:
        validation_labels = full_labels[k:data_samples,1:50,:]
    else:
        validation_labels = full_labels[k:data_samples,1:50,output_index]
    
#     print(training_data.shape)
#     print(training_labels.shape)
#     print(validation_data.shape)
#     print(validation_labels.shape)

    training_dataset = TensorDataset(torch.Tensor(training_data), torch.Tensor(training_labels))
    validation_dataset = TensorDataset(torch.Tensor(validation_data), torch.Tensor(validation_labels))

    return training_dataset, validation_dataset
# get_data_from_mat(DATA_PATH)

(4352, 49, 9)
(4352, 49, 2)
(1120, 49, 9)
(1120, 49, 2)


(<torch.utils.data.dataset.TensorDataset at 0x7ff9f95ffd90>,
 <torch.utils.data.dataset.TensorDataset at 0x7ff9f95ffc40>)

### Training Method
* Model - Model initialized based on classes above
* Save_Filepath - Where you want to save the model to. Should end with a .pt or .pth extension. This is how you are able to load the model later for testing, etc.
* training_loader - dataloader iterable with training dataset samples
* validation_loader - dataloader iterable with validation dataset samples

In [6]:
def train_model(model,save_filepath,training_loader,validation_loader,epochs,count):
    
    epochs_list = []
    train_loss_list = []
    val_loss_list = []
    training_len = len(training_loader.dataset)
    validation_len = len(validation_loader.dataset)

    #splitting the dataloaders to generalize code
    data_loaders = {"train": training_loader, "val": validation_loader}

    """
    This is your optimizer. It can be changed but Adam is generally used. 
    Learning rate (alpha in gradient descent) is set to 0.001 but again 
    can easily be adjusted if you are getting issues

    Loss function is set to Mean Squared Error. If you switch to a classifier 
    I'd recommend switching the loss function to nn.CrossEntropyLoss(), but this 
    is also something that can be changed if you feel a better loss function would work
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_func = nn.MSELoss()
#     loss_func = nn.L1Loss()
    decay_rate = 0.93 #decay the lr each step to 93% of previous lr
    lr_sch = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)

    total_start = time.time()

    """
    You can easily adjust the number of epochs trained here by changing the number in the range
    """
    for epoch in tqdm(range(epochs), position=0, leave=True):
        start = time.time()
        train_loss = 0.0
        val_loss = 0.0
        temp_loss = 100000000000000.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            for i, (x, y) in enumerate(data_loaders[phase]):  
                output = model(x)     
#                 print(output.size())
#                 print(y.size())
                loss = loss_func(torch.squeeze(output), torch.squeeze(y))  
                #backprop             
                optimizer.zero_grad()           
                if phase == 'train':
                    loss.backward()
                    optimizer.step()                                      

                #calculating total loss
                running_loss += loss.item()
            
            if phase == 'train':
                train_loss = running_loss
                lr_sch.step()
            else:
                val_loss = running_loss

        end = time.time()
        # shows total loss
        if epoch%5 == 0:
            print('[%d, %5d] train loss: %.6f val loss: %.6f' % (epoch + 1, i + 1, train_loss, val_loss))
#         print(end - start)
        
        #saving best model
        if val_loss < temp_loss:
            torch.save(model, save_filepath)
            temp_loss = val_loss
        epochs_list.append(epoch)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
    total_end = time.time()
#     print(total_end - total_start)
    #Creating loss csv
    loss_df = pds.DataFrame(
        {
            'epoch': epochs_list,
            'training loss': train_loss_list,
            'validation loss': val_loss_list
        }
    )
    # Writing loss csv, change path to whatever you want to name it
    lf = ('losses/losses_' + str(MODEL) + '_' 
          + OUTPUT + '_' + str(FRONT_TIME) + '_' 
          + str(T_END) + str(count) + '_fullin.csv')
    loss_df.to_csv(lf, index=None)
    return train_loss_list, val_loss_list

### R2 Scoring
* Model - same model as sent to train_model
* testing_dataloader - whichever dataloader you want to R2 Score

In [7]:
def r2_score_eval(model, testing_dataloader):
    output_list = []
    labels_list = []
    for i, (x, y) in enumerate(testing_dataloader):
        output = model(x) 
        output_list.append(np.transpose(output.detach().cpu().numpy()))
        labels_list.append(np.transpose(y.detach().cpu().numpy()))
    output_list = np.transpose(np.hstack(output_list))
    labels_list = np.transpose(np.hstack(labels_list)).reshape((-1,output_size))
    print(output_list.shape)
    print(labels_list.shape)
    print(r2_score(np.squeeze(labels_list), output_list))
    return output_list, labels_list

### Program Start

In [8]:
model1 = MODEL(input_size,hidden_size,output_size,batch_size,num_layers,batch_first,dropout)
model2 = MODEL(input_size,hidden_size,output_size,batch_size,num_layers,batch_first,dropout)

pnfr_training_dataset, pnfr_validation_dataset = get_data_from_mat(DATA_PATH) #retrieve PN FR
itnfr_training_dataset, itnfr_validation_dataset = get_data_from_mat(DATA_PATH, 1) #retrieve ITN FR

# Turn datasets into iterable dataloaders
pnfr_training_loader = DataLoader(dataset=pnfr_training_dataset,batch_size=batch_size,shuffle=True)
pnfr_validation_loader = DataLoader(dataset=pnfr_validation_dataset,batch_size=batch_size)

itnfr_training_loader = DataLoader(dataset=itnfr_training_dataset,batch_size=batch_size,shuffle=True)
itnfr_validation_loader = DataLoader(dataset=itnfr_validation_dataset,batch_size=batch_size)


p1 = 'models/' + str(MODEL) + '_' + OUTPUT + '_' + str(FRONT_TIME) + '_' + str(T_END) + str(0) + '_fullin.pth'
pnfr_training_loss, pnfr_validation_loss = train_model(model1,p1,pnfr_training_loader,
                                                       pnfr_validation_loader,epochs,0)
p2 = 'models/' + str(MODEL) + '_' + OUTPUT + '_' + str(FRONT_TIME) + '_' + str(T_END) + str(1) + '_fullin.pth'
itnfr_training_loss, itnfr_validation_loss = train_model(model2,p2,itnfr_training_loader,
                                                         itnfr_validation_loader,epochs,1)


# model1 = torch.load(p1)
# model1.eval()
# pnfr_t_output_list, pnfr_t_labels_list = r2_score_eval(model1, pnfr_training_loader)
# pnfr_v_output_list, pnfr_v_labels_list = r2_score_eval(model1, pnfr_validation_loader)

# model2 = torch.load(p2)
# model2.eval()
# itnfr_t_output_list, itnfr_t_labels_list = r2_score_eval(model2, itnfr_training_loader)
# itnfr_v_output_list, itnfr_v_labels_list = r2_score_eval(model2, itnfr_validation_loader)

(4352, 49, 9)
(4352, 49)
(1120, 49, 9)
(1120, 49)
(4352, 49, 9)
(4352, 49)
(1120, 49, 9)
(1120, 49)


  0%|          | 0/75 [00:00<?, ?it/s]

[1,    35] train loss: 2.183337 val loss: 0.536636


KeyboardInterrupt: 

In [ ]:
model1 = torch.load(p1)
model1.eval()
pnfr_t_output_list, pnfr_t_labels_list = r2_score_eval(model1, pnfr_training_loader)
pnfr_v_output_list, pnfr_v_labels_list = r2_score_eval(model1, pnfr_validation_loader)
# print(pnfr_t_output_list.shape)

model2 = torch.load(p2)
model2.eval()
itnfr_t_output_list, itnfr_t_labels_list = r2_score_eval(model2, itnfr_training_loader)
itnfr_v_output_list, itnfr_v_labels_list = r2_score_eval(model2, itnfr_validation_loader)

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=2)
fig.tight_layout()
ax[0,0].plot(range(epochs), pnfr_training_loss)
ax[0,0].set_title('Validation Loss')
ax[0,0].set_ylabel('Loss')
ax[0,0].set_xlabel('Epoch')

ax[0,1].plot(range(epochs), pnfr_validation_loss)
ax[0,1].set_title('Training Loss')
ax[0,1].set_ylabel('Loss')
ax[0,1].set_xlabel('Epoch')


ax[1,0].plot(np.arange(pnfr_v_labels_list.shape[1]), pnfr_v_labels_list[0,:], color='blue')
ax[1,0].plot(np.arange(pnfr_v_labels_list.shape[1]), pnfr_v_output_list[0,:], color='red')
ax[1,0].set_title('Validation PN FR for Sample 0')
ax[1,0].set_ylabel('PN FR')
ax[1,0].set_xlabel('Time')

ax[1,1].plot(np.arange(pnfr_t_labels_list.shape[1]), pnfr_t_labels_list[0,:], color='blue')
ax[1,1].plot(np.arange(pnfr_t_labels_list.shape[1]), pnfr_t_output_list[0,:], color='red')
ax[1,1].set_title('Training PN FR for Sample 0')
ax[1,1].set_ylabel('PN FR')
ax[1,1].set_xlabel('Time')

ax[2,0].plot(np.arange(itnfr_v_labels_list.shape[1]), itnfr_v_labels_list[0,:], color='blue')
ax[2,0].plot(np.arange(itnfr_v_labels_list.shape[1]), itnfr_v_output_list[0,:], color='red')
ax[2,0].set_title('Validation ITN FR for Sample 0')
ax[2,0].set_ylabel('ITN FR')
ax[2,0].set_xlabel('Time')

ax[2,1].plot(np.arange(itnfr_t_labels_list.shape[1]), itnfr_t_labels_list[0,:], color='blue')
ax[2,1].plot(np.arange(itnfr_t_labels_list.shape[1]), itnfr_t_output_list[0,:], color='red')
ax[2,1].set_title('Training ITN FR for Sample 0')
ax[2,1].set_ylabel('ITN FR')
ax[2,1].set_xlabel('Time')

plt.show()